In [1]:
import pandas as pd
import numpy as np
import eng_to_ipa as ipa
import panphon as panphon

In [59]:
import multiprocessing

def _apply(pool_map_args):
    df = pool_map_args[0]
    func = pool_map_args[1]
    args = pool_map_args[2]
#     print(args)
    
    return df.apply(func,axis = 1, args = args)

def parallel_apply(df, func, args):
    #指定主機有多少CPU，並把df 平均分配size
    num_processes = 32
    chunk_size = int(df.shape[0]/num_processes)

    #將df 拆解成 num_processes 個 chunk 的df list
    chunks = [df.ix[df.index[i:i + chunk_size]] for i in range(0, df.shape[0], chunk_size)]
    pool_map_args = [(df, func, args) for df in chunks]
    pool = multiprocessing.Pool(processes = num_processes)
    result = pool.map(_apply, pool_map_args)
    result = pd.concat([i for i in result])
    pool.terminate()
    pool.join()
    return result

In [36]:
# READ MY DATA INTO CSV FILE
msadata = pd.read_csv('../MY_TESTDATA.csv', parse_dates = True)

In [37]:
msadata['FN1_sonin'] = msadata['FN1_sonin'].fillna("-1")
msadata['FN2_sonin'] = msadata['FN2_sonin'].fillna("-1")

In [56]:
msadata[['FN1_sonin','FN2_sonin']][:1913]

,FN1_sonin,FN2_sonin
0,H,S
1,Y,F
2,S,S
3,K,S
4,Kh,Th
...,...,...
1908,K,Y
1909,H,Ch
1910,W,K
1911,Y,L


In [57]:
def convertFN1S(row):
    return ipa.convert(row['FN1_sonin'])
    
def convertFN2S(row):
    return ipa.convert(row['FN2_sonin'])

In [60]:
msadata['FN1_sonin_IPA'] = parallel_apply(msadata, convertFN1S, args = ())

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated


In [61]:
msadata['FN2_sonin_IPA'] = parallel_apply(msadata, convertFN2S, args = ())

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated


In [62]:
msadata[['FN1_sonin','FN1_sonin_IPA','FN2_sonin','FN2_sonin_IPA']]

,FN1_sonin,FN1_sonin_IPA,FN2_sonin,FN2_sonin_IPA
0,H,eʧ,S,ɛs
1,Y,waɪ,F,ɛf
2,S,ɛs,S,ɛs
3,K,ke,S,ɛs
4,Kh,kh*,Th,ˈtiˈeʧ
...,...,...,...,...
30560,-1,1,K,ke
30561,J,ʤe,X,ɛks
30562,Y,waɪ,K,ke
30563,H,eʧ,Sz,sz*


In [63]:
# SAVE NEW MSA DATA INTO CSV FILE
msadata.to_csv('MSA_SEP_IPA12.csv', index = False)